# Step 1: Import lib

In [4]:
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch as torch # core lib for deep learning methods
import time as time   # maybe I will use time to test how long a piece of code takes
import numpy as np    # it's likely that we use numpy to perform matrix manipulation
import cv2 as cv      # image I/O
import os as os

# Step 2: Build custom class for the dataset
* ref: https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
* Custom Dataset needs at least the following 3 methods:
    1. `__init__`: This function is run once when instantiating the Dataset object.
    2. `__len__`: This function returns the number of samples in our dataset.
        * At least, dataloader will need to call this function.
    3. `__getitem__`: This function loads and returns a sample from the dataset at the given index (i.e., `idx`).

In [30]:
np.random.random((3,3,5)).shape

(3, 3, 5)

In [31]:
from torch.utils.data import Dataset

class CAT_DOG_DATASET(Dataset):
    
    def __init__(self, img_dir, img_size, x_transform=None, y_transform=None):
        
        self.img_dir = img_dir
        self.img_size = img_size
        self.x_transform = x_transform
        self.y_transform = y_transform
        
        self.imgs = os.listdir(self.img_dir)
        self.len = len(self.imgs)
        self.ans_list = list()
        self.img_path_list = list()
        self.__loop_through_dir__()

    def __len__(self):
        return self.len
    
    def __getitem__(self, idx):
        
        # input: image
        img_path = self.img_path_list[idx]
        image = cv.imread(img_path)
        image = cv.resize(image, (img_size, img_size))
        image = image / 255.
        if self.x_transform:
            image = self.x_transform(image)
        
        # channel first
        image = image.reshape(3, img_size, img_size)
        
        # output: label
        label = self.ans_list[idx]
        if self.y_transform:
            label = self.y_transform(label)
        
        return image, label
    
    def __loop_through_dir__(self,):
            
            for img_name in tqdm(self.imgs):
                
                img_path = self.img_dir + img_name
                self.img_path_list.append(img_path)
                
                ans = img_name.split(".")[0]
                if ans == "cat":
                    self.ans_list.append(0)
                else: # ans == "dog"
                    self.ans_list.append(1)


# Step 3: Setup dataloader

In [32]:
train_dir_path = "../../Dataset/Cat_and_Dog/train/train/"
#os.listdir("../../Dataset/Cat_and_Dog/train/train/")

In [33]:
from torch.utils.data import DataLoader

train_dir_path = "../../Dataset/Cat_and_Dog/train/train/"
img_size = 256

tran_dataset = CAT_DOG_DATASET(img_dir=train_dir_path, img_size=img_size)
tran_loader = DataLoader(tran_dataset, batch_size=64, shuffle=True)

#dataset_test = CAT_DOG_DATASET(img_dir="./test")
#test_loader = DataLoader(dataset_test, batch_size=64)

100%|███████████████████████████████████████| 25000/25000 [00:00<00:00, 1071878.64it/s]


# Step 4: Get device for training
* ref: https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

In [34]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


# Step 5: Define the class (for model)
* `nn.Conv2d`: https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html
* `nn.Linear`: https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear
* [calculator](https://madebyollin.github.io/convnet-calculator/) for `width`, `height`, `channels`, `stride`, `padding`

In [57]:
from torch import nn

class CNN_v1(nn.Module):
    
    def __init__(self, img_size): # I add one more param here, i.e., img_size, for changing CNN structure auto
        super(CNN_v1, self).__init__()
        
        self.img_size = img_size # assume (B, C=3, H=256, W=256)
        
        self.cspec = [3, 64, 32, 16] # cspec stands for conv spec
        self.fspec = [128, 64, 1] # fspec stands for fully connected layer spec

        self.repeat_conv = nn.Sequential(

            # 換換不同的寫法 v1
            nn.Conv2d(
                in_channels = self.cspec[0],
                out_channels = self.cspec[1],
                kernel_size = (3, 3), # this could be tuple, i.e., (3,3), or just integer i.e., 3.
                stride = 2, # based on the calculator mentioned above, this setting will make spatial size half
                padding = 1 # i.e., 256 x 256 -> 128 x 128 
            ), # (B, C=64, H=128, W=128)
            
            nn.MaxPool2d(kernel_size=2), # (B, C=64, H=64, W=64)

            # 換換不同的寫法 v2
            nn.Conv2d(in_channels=self.cspec[1], out_channels=self.cspec[2], kernel_size=(3, 3), stride=2, padding=1), # (B, C=32, H=32, W=32)
            
            nn.MaxPool2d(kernel_size=2), # (B, C=32, H=16, W=16)
            
            nn.Conv2d(in_channels=self.cspec[2], out_channels=self.cspec[3], kernel_size=(3, 3), stride=2, padding=1), # (B, C=16, H=8, W=8)
        )

        self.flatten = nn.Flatten()

        # 需要優化 #
        C = 16
        H = 8
        W = H # assume square
        self.repeat_dense = nn.Sequential(
            nn.Linear(in_features= C*H*W, out_features=self.fspec[0]),
            nn.Linear(in_features= self.fspec[0], out_features=self.fspec[1]),
            nn.Linear(in_features= self.fspec[1], out_features=self.fspec[2]),
        )
        
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, img):
        feature_map = self.repeat_conv(img)
        features = self.flatten(feature_map)
        logits = self.repeat_dense(features)
        probabilities = self.sigmoid(logits)
        return probabilities
    
def test():
    batch_size = 10
    channel_num = 3
    img_size = 256

    CNN_model = CNN_v1(img_size=img_size)
    
    # Usually use torch.randn to test if model can work as expected
    input_img = torch.randn(batch_size, channel_num, img_size, img_size) # (batch_size, channel, width, height)
    
    output_val= CNN_model(input_img)
    
    assert output_val.detach().numpy().shape == (batch_size, 1)

# Step 6: Hyperparamter setting

In [58]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

# Step 7: Loss function
* `BCELoss`: https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html

In [59]:
# binary cross entropy loss
bce_loss = torch.nn.BCELoss()

# Step 8: Setup Model

In [60]:
CNN_model = CNN_v1(img_size=img_size)

# Step 9: Choose optimizer

In [61]:
sgd_optimizer = torch.optim.SGD(CNN_model.parameters(), lr=learning_rate)

# Step 9: Optimization process
* ref: https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html
* ref2: https://discuss.pytorch.org/t/expected-object-of-scalar-type-double-but-got-scalar-type-float-for-argument-3-mat1-in-call-to-th-addmm/109332

In [66]:
def train_loop(dataloader, model, loss_fn, optimizer):
    
    size = len(dataloader.dataset)
    
    for batch, (X, y) in tqdm(enumerate(dataloader)):
        
        # derive prediction
        pred = model(X.float()) # Wondering about X.float()? check ref2
        loss = loss_fn(pred, y.float())
        
        # backward propagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [65]:
for t in range(epochs):
    print(f"Epoch {t+1}\n----------------------------")
    train_loop(tran_loader, CNN_model, bce_loss, sgd_optimizer)
print("Done")

Epoch 1
----------------------------


1it [00:00,  1.55it/s]

loss: 0.692704 [    0/  391]


101it [01:04,  1.58it/s]

loss: 0.693342 [ 6400/  391]


201it [02:09,  1.53it/s]

loss: 0.688099 [12800/  391]


213it [02:18,  1.54it/s]


KeyboardInterrupt: 